In [10]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import json
import pandas as pd
from collections import OrderedDict

In [11]:
title = []
rate = []
year = []
age_restriction = []
typee = []
origin = []
duration = []
imdb_rank = []
keshvar_sazandeh=[]

In [12]:
for i in range(20):
    url = "https://filmnet.ir/api-v2/video-contents?offset=" + str(24*i) + "&count=24&order=latest&query=&types=single_video&types=series&types=video_content_list"
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    page = session.get(url)
    j = json.loads(page.text)
    for k in range(24):
        #print(k)
        title.append(j['data'][k]['title'])
        rate.append(round(j['data'][k]['rate'],2))
        try:
            year.append(j['data'][k]['year'])
        except:
            year.append('---------')
        age_restriction.append(j['data'][k]['age_restriction'])
        typee.append(j['data'][k]['type'])
        origin.append(j['data'][k]['original_name'])
        try:
            duration.append(j['data'][k]['duration'])
        except KeyError:
            duration.append('---------')
        try:
            imdb_rank.append(j['data'][k]['imdb_rank_percent'])
        except KeyError:
            imdb_rank.append('Not Mentioned')
            
        categories=j['data'][k]['categories']
        territory_index = next((index for (index, d) in enumerate(categories) if d["type"] == "territory"), None)
        if territory_index == None:
            keshvar_sazandeh.append('Unknown')
        else:
            countries= " - ".join([ii['title'] for ii in categories[territory_index]['items']]).strip()
            keshvar_sazandeh.append(countries)

In [78]:
categories=j['data'][11]['categories']
#territory_index = next((index for (index, d) in enumerate(categories) if d["type"] == "territory"), None)
#countries= " - ".join([ii['title'] for ii in categories[territory_index]['items']]).strip()
#keshvar_sazandeh.append(countries)
#territory_index
categories

[{'type': 'default', 'items': [{'title': 'پشت صحنه'}]}]

In [79]:
url = "https://filmnet.ir/api-v2/video-contents?offset=24&count=24&order=latest&query=&types=single_video&types=series&types=video_content_list"
page = requests.get(url)
j = json.loads(page.text)
categories=j['data'][11]['categories']
print(categories)
#territory_index = next((index for (index, d) in enumerate(categories) if d["type"] == "territory"), None)
#print(territory_index)
#[ii['title'] for ii in categories[territory_index]['items']]
#",".join([ii['title'] for ii in categories[territory_index]['items']]).strip()


[{'type': 'genre', 'items': [{'title': 'درام'}, {'title': 'جنایی'}, {'title': 'بیوگرافی'}]}, {'type': 'territory', 'items': [{'title': 'آمریکا'}]}, {'type': 'default', 'items': [{'title': 'خارجی'}]}]


In [55]:
len(duration)

12

In [13]:
movies_info= OrderedDict()
movies_info['نام اثر'] = title
movies_info['رأی'] = rate
movies_info["رأی در IMDB"] = imdb_rank
movies_info['سال ساخت'] = year
movies_info['محدودیت سنی'] = age_restriction
movies_info['نوع'] = typee
movies_info['نام انگلیسی'] = origin
movies_info['زمان'] = duration
movies_info['کشور سازنده'] = keshvar_sazandeh
movies_info_df= pd.DataFrame(movies_info)

In [15]:
movies_info_df
movies_info_df.index += 1

In [16]:
movies_info_df

,نام اثر,رأی,رأی در IMDB,سال ساخت,محدودیت سنی,نوع,نام انگلیسی,زمان,کشور سازنده
1,شبی که ماه کامل شد,45.00,68,1397,not_set,single_video,When the Moon Was Full,02:10:57,‌ ایران
2,وینچسترها,25.00,63,2022,fifteen,series,The Winchesters,---------,آمریکا
3,چشم انداز زیبا,25.00,66,2022,not_set,single_video,Nice View,01:45:21,چین
4,شرور,25.00,73,2022,eighteen,series,The Bad Guy,---------,ایتالیا
5,نارویک: اولین شکست هیتلر,25.00,66,2022,fifteen,single_video,Narvik: Hitler's First Defeat,01:43:10,نروژ
...,...,...,...,...,...,...,...,...,...
476,روزی روزگاری سینما,39.69,0,1392,not_set,single_video,,01:12:37,‌ ایران
477,اساتید لگو,48.00,74,2020,thirteen,series,Lego Masters,---------,آمریکا
478,داستان های جدای,50.00,86,2022,thirteen,series,Star Wars: Tales of the Jedi,---------,آمریکا
479,دره مومین,47.00,72,2019,seven,series,Moominvalley,---------,انگلستان - فنلاند


In [17]:
movies_info_df.to_csv("filmnet_data_csv")

In [18]:
filmnet_data_csv

',نام اثر,رأی,رأی در IMDB,سال ساخت,محدودیت سنی,نوع,نام انگلیسی,زمان,کشور سازنده\r\n1,شبی که ماه کامل شد,45.0,68,1397,not_set,single_video,When the Moon Was Full,02:10:57,\u200c ایران\r\n2,وینچسترها,25.0,63,2022,fifteen,series,The Winchesters,---------,آمریکا\r\n3,چشم انداز زیبا,25.0,66,2022,not_set,single_video,Nice View,01:45:21,چین\r\n4,شرور,25.0,73,2022,eighteen,series,The Bad Guy,---------,ایتالیا\r\n5,نارویک: اولین شکست هیتلر,25.0,66,2022,fifteen,single_video,Narvik: Hitler\'s First Defeat,01:43:10,نروژ\r\n6,چراغ های قرمز,25.0,62,2012,adult,single_video,Red Lights,01:52:30,کانادا  -  اسپانیا\r\n7,مارکس می\u200cتواند صبر کند,25.0,76,---------,eighteen,single_video,Marx Can Wait,01:27:15,ایتالیا\r\n8,ارتفاع,50.0,72,2022,thirteen,single_video,Uunchai,02:32:30,هند\r\n9,پلنگ سیاه: واکاندا تا ابد,44.0,72,2022,thirteen,single_video,Black Panther: Wakanda Forever,02:33:18,آمریکا\r\n10,ژان کلود ون جانسون,50.0,75,2016,fifteen,series,Jean-Claude Van Johnson,---------,آمریکا\r\n11,بلک\u200cوی

In [19]:
import zlib
import zipfile


def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)


file_names = ["filmnet_data_csv", "Filmnet_bs4.ipynb"]
compress(file_names)

File Paths:
['filmnet_data_csv', 'Filmnet_bs4.ipynb']


FileNotFoundError: [WinError 2] The system cannot find the file specified: './filmnet_data_csv'